In [1]:
#librerie necessarie

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv
from langchain_core.runnables import Runnable, RunnablePassthrough, RunnableParallel, RunnableLambda
from IPython.display import JSON
import json
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext watermark

In [ ]:

# Carica le variabili di ambiente dal file .env
load_dotenv()

# Ora puoi accedere alle variabili di ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#llm studio

model = ChatOpenAI(
    base_url = "http://172.17.0.1:1234/v1",
    temperature = 0,
    api_key = "not-needed",
    model_name ="llama3",
)

In [2]:
#ollama

model = ChatOpenAI(
    base_url = "http://localhost:11434/v1",
    temperature = 0,
    api_key = "not-need",
    model_name ="llama3",
)

In [ ]:
#openai

model = ChatOpenAI(
    emperature = 0,
     model_name ="gpt-4o-mini",
)

In [3]:
pdfd="/home/nugh75/git-repository/env1/Algorithm_of_Thoughts-_Enhancing_Exploration_of_Ideas_in_Large_Language_Models.pdf"

In [4]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = pdfd

# Call the partition_pdf function
# Returns a List[Element] present in the pages of the parsed pdf document
elements = partition_pdf(filename)

# Now, elements is a list of all elements present in the pages of the parsed pdf document

In [ ]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = pdfd

# Call the partition_pdf function
# Returns a List[Element] present in the pages of the parsed pdf document
all_documents = partition_pdf(filename)

# Now, elements is a list of all elements present in the pages of the parsed pdf document

In [5]:
elements

In [ ]:
titolo = elements[0]
print(titolo)

In [ ]:
autore = elements[2]
print(autore)

In [5]:
# Supponiamo che `elements` sia già definito e contenga una lista di oggetti con il metodo `to_dict()`

# Converte ogni elemento in un dizionario usando il metodo `to_dict()`
element_dicts = [el.to_dict() for el in elements]

# Estrae solo il campo "text" di ogni dizionario
texts = [element["text"] for element in element_dicts]

# Converte la lista di testi in una stringa JSON ben formattata
testo= json.dumps(texts, indent=2)

# Stampa la stringa JSON formattata
print(testo)

[
  "3 2 0 2",
  "p e S 8 2",
  "] L C . s c [",
  "2 v 9 7 3 0 1 . 8 0 3 2 : v i X r a",
  "Algorithm of Thoughts: Enhancing Exploration of Ideas in Large Language Models",
  "Bilgehan Sel, Ahmad Al-Tawaha, Vanshaj Khattar, Ruoxi Jia, and Ming Jin",
  "Virginia Tech",
  "Abstract",
  "Current literature, aiming to surpass the \u201cChain-of-Thought\u201d approach, often resorts to an external modus operandi in- volving halting, modifying, and then resuming the genera- tion process to boost Large Language Models\u2019 (LLMs) rea- soning capacities. This mode escalates the number of query requests, leading to increased costs, memory, and computa- tional overheads. Addressing this, we propose the Algorithm of Thoughts\u2014a novel strategy that propels LLMs through algorithmic reasoning pathways, pioneering a new mode of in-context learning. By employing algorithmic examples, we exploit the innate recurrence dynamics of LLMs, expand- ing their idea exploration with merely one or a few qu

In [ ]:
# Supponiamo che `elements` sia già definito e contenga una lista di oggetti con il metodo `to_dict()`

# Converte ogni elemento in un dizionario usando il metodo `to_dict()`
element_dicts = [el.to_dict() for el in elements]

# Filtra solo gli elementi di tipo "Title" ed estrae il campo "text"
titles = [element["text"] for element in element_dicts if element["type"] == "Title"]

# Converte la lista di testi dei titoli in una stringa JSON ben formattata
indice = json.dumps(titles, indent=2)

# Stampa la stringa JSON formattata
print(indice)

In [ ]:
Testo_da_riassumere=testo

In [ ]:
#Esempio di template prompt

prompt = ChatPromptTemplate.from_template("Riassumi questo testo{testou} ; {argomento}, Rispondi in italiano.")
output_parser = StrOutputParser()
chain = prompt | model | output_parser
answer = chain.invoke({"argomento": "La linguistica"})
print(answer)

In [ ]:
#Esempio di template prompt

# Definisci il prompt template
prompt = ChatPromptTemplate.from_template(
    """Riassumi questo testo {testou} il cui titolo è {titolou}, il cui autore è {autoreu}, il cui indice è {indice}. Dividi il riassunto i queste ssezioni: quadro teorico, obiettivi, metodologia, risultati e conclusioni. Queste sezionio saranno i paragrafi del riassunto. Il riassunto deve essere discorsivo e ogni parte deve avere almeno cinque capoversi. Inserisci esempi, definizioni e loro spigazione.  All'inizio del riassunto fai una piccola scheda con titolo, autore e concetti più importanti. Non fare un elenco puntato. Agisci come un ricercatore. Rispondi in italiano. dopo aver fatto il riassunto e cerca di migliorarlo ulteriormente. Alla fine metti i riferimenti bibliografici che si trovano nella sezione bibliografia in formato APA. Se parole sono tecniche e centrarli nel testo lasciale in inlgese"""
)

# Crea l'output parser
output_parser = StrOutputParser()

# Crea la catena di trasformazione
chain = prompt | model | output_parser

# Definisci i parametri di input
params = {
    "testou": testo,
    "titolou": titolo,
    "autoreu": autore,
    "indice": indice
}

# Esegui la catena e ottieni la risposta
answer = chain.invoke(params)

# Stampa la risposta
print(answer)

In [ ]:
#Caricamento file da cartella

#percorso della cartella contenente i PDF
folder_path = 'pdfs'

# Lista per memorizzare tutti i documenti caricati
all_documents = []

# Iterare su tutti i file nella cartella
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path=pdf_path)
        documents = loader.load()
        all_documents.extend(documents)

In [ ]:
len(all_documents)

In [ ]:
print(all_documents)

In [ ]:
print(all_documents[0])

In [ ]:
all_documents[16].metadata

In [ ]:
len(all_documents[0].page_content)

In [ ]:
# Stampare i nomi dei file PDF caricati
print("PDF caricati:")
for pdf in all_documents:
    print(pdf)

In [ ]:
print(all_documents[0].page_content[0:1000])

In [ ]:
all_documents=testo

In [14]:
# Carica il PDF
loader = PyPDFLoader(pdfd)

# Carica i documenti
documents = loader.load()

# Assicurati che ogni documento abbia l'attributo 'page_content'
for doc in documents:
    print(doc.page_content)

Algorithm of Thoughts: Enhancing Exploration of Ideas
in Large Language Models
Bilgehan Sel, Ahmad Al-Tawaha, Vanshaj Khattar, Ruoxi Jia, and Ming Jin
Virginia Tech
Abstract
Current literature, aiming to surpass the “Chain-of-Thought”
approach, often resorts to an external modus operandi in-
volving halting, modifying, and then resuming the genera-
tion process to boost Large Language Models’ (LLMs) rea-
soning capacities. This mode escalates the number of query
requests, leading to increased costs, memory, and computa-
tional overheads. Addressing this, we propose the Algorithm
of Thoughts —a novel strategy that propels LLMs through
algorithmic reasoning pathways, pioneering a new mode of
in-context learning. By employing algorithmic examples, we
exploit the innate recurrence dynamics of LLMs, expand-
ing their idea exploration with merely one or a few queries.
Our technique outperforms earlier single-query methods and
stands on par with a recent multi-query strategy that employs
an e

In [13]:
# Define the content and the file path
content = testo
file_path = "testo.txt"

# Create and write to the file
with open(file_path, "w") as file:
    file.write(content)

print(f"File created successfully at {file_path}")

File created successfully at testo.txt


In [16]:
 # Read the text content from the file
loader = TextLoader(file_path)
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =500,
    chunk_overlap=200
)
splits =text_splitter.split_documents(documents)
for sp in splits:
    if (len(sp.page_content) <100):
        splits.remove(sp)

In [18]:
print ("Numero di chunks:",len(splits))

Numero di chunks: 354


In [6]:
print("Metadata :", splits[0].metadata)
print("Dimenzione del chunk:", len(splits[1].page_content), "caratteri")
print("Contenuto del chunk: \n" )
print(splits[1].page_content)

NameError: name 'splits' is not defined

In [17]:
#Select Embeddings model
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(

model ="nomic-embed-text" 
)

In [20]:
#Select Embeddings model questo funziona

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [18]:
cartella="algoritmop"

In [ ]:
import os
from langchain_community.vectorstores import FAISS

if os.path.exists(cartella):
    #Carica indice FAISS cartella corrente attuale
    faiss_index=FAISS.load_local(
        cartella,
        embeddings,
        allow_dangerous_deserialization=True
    )
else:
    #crea indice FAISS dei chunk nella cartella attuale
    faiss_index = FAISS.from_documents(
        splits,
        embeddings
    )
    faiss_index.save_local(cartella)


In [20]:
import os
from langchain_community.vectorstores import FAISS

if os.path.exists(cartella):
    #Carica indice FAISS cartella corrente attuale
    faiss_index=FAISS.load_local(
        cartella,
        embeddings,
        allow_dangerous_deserialization=True
    )
else:
    #crea indice FAISS dei chunk nella cartella attuale
    faiss_index = FAISS.from_documents(
        docs,
        embeddings
    )
    faiss_index.save_local(cartella)

In [21]:
retriever = faiss_index.as_retriever(

    search_type="similarity",
    search_kwargs={"k": 4} 
)

In [ ]:
from langchain import hub
prompt =hub.pull("rlm/rag-prompt")

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_template("Sei un assistente preciso e attento ; Rispondi a questa domanda in italiano: {question}, considra il seguente contesto {context}.")

In [ ]:
#Esempio di template prompt

# Definisci il prompt template
prompt = ChatPromptTemplate.from_template(
    "Riassumi questo testo {testou} il cui titolo è {titolou}, il cui autore è {autoreu}, il cui indice è {indice}. Dividi le sezioni del riassunto secondo l'indice. Il riassunto deve essere discorsivo e ogni parte deve avere almeno due capoversi. Nella sezione reference o biografia riporta i dati biografici. Inserisci esempi, definizioni e loro spigazione. Non dire in questa sezione parla di o cose del genere, ma spiega in parole più semplice quello che viene scritto, cercando di non perdere il significato originalie, devi essere informativo. Fai tipo un scheda titolo, autore e riassutnto. Non fare un elenco puntato. Agisci come un ricercatore. Rispondi in italiano."
)

# Crea l'output parser
output_parser = StrOutputParser()

# Crea la catena di trasformazione
chain = prompt | model | output_parser

# Definisci i parametri di input
params = {
    "testou": testo,
    "titolou": titolo,
    "autoreu": autore,
    "indice": indice
}

# Esegui la catena e ottieni la risposta
answer = chain.invoke(params)

# Stampa la risposta
print(answer)

In [ ]:
#Esempio di template prompt

# Definisci il prompt template
prompt = ChatPromptTemplate.from_template(
    "Riassumi questo testo {testou} il cui titolo è {titolou}, il cui autore è {autoreu}, il cui indice è {indice}. Dividi le sezioni del riassunto secondo l'indice. Il riassunto deve essere discorsivo e ogni parte deve avere almeno due capoversi. Nella sezione reference o biografia riporta i dati biografici. Inserisci esempi, definizioni e loro spigazione. Non dire in questa sezione parla di o cose del genere, ma spiega in parole più semplice quello che viene scritto, cercando di non perdere il significato originalie, devi essere informativo. Fai tipo un scheda titolo, autore e riassutnto. Non fare un elenco puntato. Agisci come un ricercatore. Rispondi in italiano."
)

# Crea l'output parser
output_parser = StrOutputParser()

# Crea la catena di trasformazione
chain = prompt | model | output_parser

# Definisci i parametri di input
params = {
    "testou": retriever | format_documents,
    "titolou": titolo,
    "autoreu": autore,
    "indice": indice
}

# Esegui la catena e ottieni la risposta
answer = chain.invoke(params)

# Stampa la risposta
print(answer)

In [23]:
def format_documents(documents):
    return "\n\n".join(doc.page_contente for doc in documents)

rag_chain = (
    {
        "context": retriever | format_documents,
        "question": RunnablePassthrough()
        
    } 
    |prompt
    |model
    |StrOutputParser()
    
    
)

In [24]:
def query(query):
    answer = rag_chain.invoke(query)
    return answer

def queryStream(query):
    for chunk in rag_chain.stream(query):
        print(chunk, end="", flush=True)

In [ ]:
print(testo)

In [ ]:
query("riasumi Algorithm_of_Thoughts-_Enhancing_Exploration_of_Ideas_in_Large_Language_Models")

In [ ]:
queryStream("Qual è il titolo dell'articolo e qual è il suo autore, che cosa sono le diacostruzione")